In [14]:
import os
#arxiv
#wikipedia
os.environ["OPENAI_API_KEY"] = <<OPENAI_API_KEY>>

In [4]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [7]:
## Used inbuilt toold for wikipedia

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [9]:
api_wraper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=api_wraper_arxiv)
arxiv.name

'arxiv'

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriver = vectordb.as_retriever() # from retriver you can query anything from here 
#retriver 
#convert retriever into tool

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002202921DF00>, search_kwargs={})

In [18]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, 'langsmit_search', 'description about tool')
retriver_tool.name

'langsmit_search'

In [22]:
tools = [wiki,arxiv,retriver_tool]

In [23]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'P:\\GEN-AI\\notebook\\.venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='langsmit_search', description='description about tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021FB242BA30>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002202921DF00>, search_kwargs={}), document

In [ ]:
## Run all the tools with agents and LLM models 
## Tools, LLM-->AgentExecutor
## langchain_groq 

In [26]:
from langchain_groq import ChatGroq
groq_api_key = <<groq_api_key>>
llm = ChatGroq(groq_api_key=groq_api_key, model_name= "Llama3-8b-8192")

In [28]:
# prompt template
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


P:\GEN-AI\notebook\.venv\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
### Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [30]:
# to Run Agent executor 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose= True)

In [31]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [32]:
agent_executor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmit_search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!
1. Install LangSmith​
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key​
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment​
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessa

{'input': 'Tell me about langsmith',
 'output': "It seems that you've already received the information about LangSmith, and I should not call the tool again for the same topic. Instead, I can provide a direct response to your initial question.\n\nLangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. LangSmith is built on top of LangChain's open-source frameworks and provides a user-friendly interface for setting up and configuring LLM applications.\n\nThe platform offers a range of features, including installation, API key creation, environment setup, and logging and evaluation of traces. LangSmith is designed to be flexible and can be used with a variety of programming languages, including Python and JavaScript.\n\nOverall, LangSmith is a powerful tool for developers who want to build and deploy LLM applications quickly an

In [33]:
agent_executor.invoke({"input":"who is prime minister of india?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Prime Minister of India'}`


Page: Prime Minister of India
Summary: The prime minister of India (ISO: Bhārata kē/kī pradhānamaṁtrī) is the head of government of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers, despit
Invoking: `wikipedia` with `{'query': 'current prime minister of india'}`


Page: Prime Minister of India
Summary: The prime minister of India (ISO: Bhārata kē/kī pradhānamaṁtrī) is the head of government of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers, despitNarendra Modi

> Finished chain.


{'input': 'who is prime minister of india?', 'output': 'Narendra Modi'}

In [34]:
agent_executor.invoke({"input":"What is the paper 1706.03762 about"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks 
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,Based on the conversation, it seems that the paper "1706.03762" is actually the paper "Attention Is All You Need" by Vaswani et al. and it's a 2017 paper introducing the transformer architecture.

> Finished chain.


{'input': 'What is the paper 1706.03762 about',
 'output': 'Based on the conversation, it seems that the paper "1706.03762" is actually the paper "Attention Is All You Need" by Vaswani et al. and it\'s a 2017 paper introducing the transformer architecture.'}